In [7]:
import numpy as np
import math
#import pylab
import matplotlib.pyplot as plt
%matplotlib notebook
#import os
#import glob
import sys
import pandas as pd
#from scipy.misc import imread, imsave
#from read_data_pandas import *

### Cp calculation method 1:

cp = 2 tau omega /(rho V^3 A_swept)

In [8]:
# Enter torque about x-axis in [N-m]
tau = 0.0000021113932  # LES time hist
tau2 = 2.387698e-06 # RANS OD extend (5k iterations)
# tau = 2.342281e-06 # RANS 10k iterations
# tau = 2.277e-06  # RANS 
omega = 824.0 #RPM
omega = omega*(2*math.pi)/60 # rpm to rad/s
Vx = .737616
# r_tip = 0.0308 # [m] this is the seed trip radius that I have been using (seems like and undershoot)
r_tip = 0.0316 # [m] this is the most accurate cone dy radius I can get
a_swept = math.pi*r_tip**2. #[m^2]
#omega = 86.18 # [rad/s]
rho = 1.18415 # [kg/m^3]
#Vx = .737616 # [m/s]
print "tau = ", tau
print "omega =", omega
print "Vx = ", Vx

Cp = (tau*omega)/(0.5 * rho * (Vx**3.) * (r_tip**2) * math.pi)
Cp2 = 2.*(tau2*omega)/(rho*Vx**3.*a_swept)
print "Cp =",Cp
print ' RANS OD extend Cp = ', Cp2

tau =  2.1113932e-06
omega = 86.2890782186
Vx =  0.737616
Cp = 0.244418008212
 RANS OD extend Cp =  0.276403461644


Read in Torque history from LES:

In [60]:
torque_dir = '/Users/jakeholden/Dropbox/MapleSeed/Thesis/Scripts/Result_calculations/LES/output/'
torque_file = 'Torque_wrt_x_via_moment_report_Monitor_Plot_seed_torquex_21561.csv'
df_torque_hist = pd.DataFrame.from_csv(torque_dir+torque_file,index_col = False)
df_torque_hist['Torque_wrt_x_via_moment_report Monitor: Torque_wrt_x_via_moment_report Monitor (N-m)'].mean()

2.111393200721068e-06

### Cp calculation method 2:

I expect this method would undershoot the Cp estimation because the conical plane where velcity is surface averaged is upstream of the rotor plane (so there is still some axial distance for flow to decelerate after the plane. Also, the surface is nominally smaller than the mapleseed tip radius, so a small region of velocity deifict may be exclude from this calculation.

Cp = 4 a (1 - a)^2

a = (U1 - U2)/U1

In [21]:
U1 = .737616 # [m/s]
U2_3k = 5.9381e-01 # RANS 3k iterations
U2 = 5.816754e-01 # RANS 10k iterations
# U2 = 5.98561e-01 # RANS 6415_RANS_v9.sim 2450 iterations
# U2 = 5.447363e-01 # RANS 64415_RANS_refine.sim 1000 iterations NOT CONVERGED
# U2 = 6.105853e-01 # RANS 121414_coning_angle_test2_revisited 2500 iterations
# U2 = 5.940952e-01 # RANS ms_1_RANS_5dia_cone 3000 iterations
# U2_les = 5.825101e-01 # LES Time Mean
U2_les = 5.8231e-01 # LES Time Mean ts = 0.584 secs
U2_od = 5.838816e-01 # RANS OD extended wall (5k iterations)

a =(U1-U2)/U1
a_emp = .257
a_3k = (U1-U2_3k)/U1
a_les =(U1-U2_les)/U1
a_od = (U1-U2_od)/U1

Cp2 = 4.0*a*(1.0-a)**2
Cp2_3k = 4.0*a_3k*(1.0-a_3k)**2
Cp2_les = 4.0*a_les*(1.0-a_les)**2
Cp2_emp = 4.0*a_emp*(1.0-a_emp)**2
Cp2_od = 4.0*a_od*(1.0-a_od)**2

print "RANS Cp = ", Cp2
print "RANS 3k iterations Cp = ", Cp2_3k
print "LES Cp = ", Cp2_les 
print "Empirical Cp = ", Cp2_emp 
print "RANS OD Cp = ", Cp2_od

print "RANS/LES Error = ", (Cp2_les - Cp2)/Cp2_les

print "a_rans_10k = ", a

RANS Cp =  0.525883567689
RANS 3k iterations Cp =  0.50540673601
LES Cp =  0.524886902941
Empirical Cp =  0.567506372
RANS OD Cp =  0.522383729278
RANS/LES Error =  -0.0018988180922
a_rans_10k =  0.211411628815


### CT calculation:

CT = (2 Fx) / (rho V^2 pi r_tip^2)

CT = 4a(1-a)

In [53]:
CT2 = 4.0*a*(1.0-a)
CT2_les = 4.0*a_les*(1.0-a_les)
CT2_emp = 4.0*a_emp*(1.0-a_emp)

print "RANS CT = ", CT2
print "LES CT = ", CT2_les
print "Empirical CT = ", CT2_emp

print "RANS a = ", a
print 'LES a = ', a_les

CT_betz = .889
print "CT error = ", (CT_betz - CT2_les)/CT_betz

RANS CT =  0.666867008067
LES CT =  0.66424931003
Empirical CT =  0.763804
RANS a =  0.211411628815
LES a =  0.210280010195
CT error =  0.252812924601


### TSR Calculation:

In [3]:
lamda = (r_tip*omega)/Vx
print lamda

3.69668617778


### Reynolds Number Calculation:

In [12]:
rho = 1.225 # [kg/m^3]
mu = 1.983e-5 # [Pa s]
max_c = .0145 # [m]
r_max_c = .95*r_tip
v_res = (Vx**2+(omega*r_max_c)**2)**.5
Re = (rho*v_res*max_c)/mu
print Re

2412.55560248


### Betz-RANS differences in performance params (section 3.4.2):

In [31]:
Cp_m1_rans = 55.42
Cp_m1_betz = 59.26
Cp_m1_error = (Cp_m1_betz - Cp_m1_rans)/Cp_m1_betz
print "Cp_m1_error = ", Cp_m1_error

Cp_m2_rans = 54.22
Cp_m2_les = 52.46
Cp_m2_betz = 59.26
Cp_m2_error = (Cp_m2_betz - Cp_m2_rans)/Cp_m2_betz
Cp_m2_les_error = (Cp_m2_betz - Cp_m2_les)/Cp_m2_betz
print "Cp_m2_error = ", Cp_m2_error
print "Cp_m2_les_error = ", Cp_m2_les_error

CT_rans = 87.0
CT_betz = 88.9
CT_error = (CT_betz - CT_rans)/CT_betz
print "CT_error = ", CT_error

a_rans = 31.3
a_betz = 33.3
a_error = (a_betz - a_rans)/a_betz
a_les_error = (a_betz - a_les*100.0)/a_betz
print "a_error = ", a_error
print "a_les_error = ", a_les_error

Cp_m1_error =  0.0647991900101
Cp_m2_error =  0.0850489368883
Cp_m2_les_error =  0.114748565643
CT_error =  0.0213723284589
a_error =  0.0600600600601
a_les_error =  0.368528497913


In [65]:
(.0442453+.03810329)/.08252794

0.997826796597613

## Maple Seed Power Output Calculation:

In [4]:
rho = 1.225 # [kg/m^3]
U2 = 5.816754e-01 # RANS 10k iterations

# Power_out = 0.5*rho*(U2**3)*a_swept

tau = 2.342281e-06 # RANS 10k iterations
Power_out = tau*omega

print 'Power output = ', Power_out, 'Watts'

Power output =  0.000202113268419 Watts


## Maximum Cp Calc with Wake Rotation

In [112]:
from scipy.optimize import fsolve

# Define the expression whose roots we want to find
L = 3.7

func = lambda a2 : L**2 - ((1.0-a2)*(1.0-4.0*a2)**2)/(1.0-3.0*a2) 

# Plot it

# a2 = np.linspace(0, 0.333, 100)

# plt.figure()
# plt.plot(func(a2), a2)
# plt.xlabel("TSR")
# plt.ylabel("a2")
# plt.grid()
# plt.show()

# Use the numerical solver to find the roots

a2_initial_guess = .32
a2_solution = fsolve(func, a2_initial_guess)

print "The solution is a2 = %f" % a2_solution
print "at which the value of the expression is %f" % func(a2_solution)

The solution is a2 = 0.331600
at which the value of the expression is 0.000000


In [113]:
x1 = 0.25
x2 = (1.0 - 3.0*a2_solution)

Cp_max_1 = (8.0/(729.0*L**2))*((64.0/5.0)*x1**5 + 72.0*x1**4 + 124.0*x1**3 + 38.0*x1**2 - 63.0*x1 - 12.0*np.log(x1) - 4.0*x1**-1.0)

Cp_max_2 = (8.0/(729.0*L**2))*((64.0/5.0)*x2**5 + 72.0*x2**4 + 124.0*x2**3 + 38.0*x2**2 - 63.0*x2 - 12.0*np.log(x2) - 4.0*x2**-1.0)

Cp_max = Cp_max_1 - Cp_max_2

print "Cp_max with wake rotation for TSR = %0.1f is Cp = %0.4f " % (L, Cp_max)

Cp_max with wake rotation for TSR = 3.7 is Cp = 0.5577 


## $C_T$ Calc from Pressure and Shear forces:

${C_T} = \frac{2 F_x}{\rho V_{ax}^2 A_{ax,swept}}$ 

${F_x}$ = Sum of forces on the blade

In [20]:
# Values from RANS (10k iteration) solution:
rho = 1.225 # [kg/m^3]
Axial_force = 8.8518E-4   # [N] Force report
# Axial_force2 = 8.8524E-4  # OD extended RANS solution
Thrust_coef = (2.*Axial_force)/(rho* U1**2 * r_tip**2 * math.pi) # N
Thrust_coef2 = (2.*Axial_force2)/(rho* U1**2 * r_tip**2 * math.pi) # N

print 'Ct = ', Thrust_coef#*math.cos(math.radians(16))
print 'Fx = ', Axial_force
print 'B-R/B for CT RANS 10k ', (.889-Thrust_coef)/.889

Ct =  0.846721923059
Fx =  0.00088518
B-R/B for CT RANS 10k  0.0475568919476


In [14]:
# Seed mass per this axial force:
m_seed = Axial_force/9.8
print 'Seed mass = ', m_seed*1000, ' grams'

Seed mass =  0.0903244897959  grams


## Basin Efficiency:

$\eta_{basin} = \frac{Power}{Thrust \cdot V}$

In [20]:
eta_b = (202E-6)/(Axial_force*U1)  # 202 micro watts is P_out for maple seed
print 'Basin Efficiency = ', eta_b

Basin Efficiency =  0.309378014535
